# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
#I split the dataset into a train and test ds
from datasets import load_dataset

import torch
from torch.utils.data import Dataset, random_split

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("michsethowusu/swati-emotions-corpus", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

for split in splits:
   dataset[split] = dataset[split].shuffle(seed=42).select(range(15000))

# View the dataset characteristics
dataset["test"]

Dataset({
    features: ['Swati', 'emotion'],
    num_rows: 15000
})

In [3]:
#i need to remove some columns not required for this task
#dataset = dataset.remove_columns(['label', 'dataType', 'communityName', 'datetime'])

# Rename 'emotion' to 'label' for easier PyTorch handling
dataset = dataset.rename_column('emotion', 'label')

# Inspect the dataset
print(dataset)

dataset["train"][3:5]

DatasetDict({
    train: Dataset({
        features: ['Swati', 'label'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['Swati', 'label'],
        num_rows: 15000
    })
})


{'Swati': ['Loko kutocinisekisa kuphepha kwakho njengafakazi, nekubakhona kwebufakazi, nekuvimbela kutsi uhocise ekwetfuleni bufakazi ngenca yalokufakwa umoya.',
  'Bantfu bafundzani ngaloku lokwenteka kulendzatjana?'],
 'label': ['neutral', 'surprise']}

In [4]:
#handling lables that are strings

unique_labels = list(set(dataset["train"]["label"]))

# Step 2: Create label2id and id2label mappings
label2id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id2label = {idx: label for label, idx in label2id.items()}


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_batch(batch):
    encodings = tokenizer(batch["Swati"], padding="max_length", truncation=True)
    encodings["label"] = [label2id[label] for label in batch["label"]]
    return encodings

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(preprocess_batch, batched=True)
    
# Inspect the available columns in the dataset
tokenized_dataset["train"]


Dataset({
    features: ['Swati', 'label', 'input_ids', 'attention_mask'],
    num_rows: 15000
})

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=7
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:
import os
import torch

checkpoint_dir = "/workspace/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Save checkpoint
checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_latest.pth")
torch.save(model.state_dict(), checkpoint_path)

# Keep only the last 3 checkpoints
checkpoints = sorted(os.listdir(checkpoint_dir), reverse=True)
if len(checkpoints) > 3:
    os.remove(os.path.join(checkpoint_dir, checkpoints[-1]))  # Delete the oldest checkpoint

In [8]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


training_args = TrainingArguments("test-trainer")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [9]:
# Show the performance of the model on the test set

trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.9242671728134155,
 'eval_accuracy': 0.32893333333333336,
 'eval_runtime': 240.4599,
 'eval_samples_per_second': 62.38,
 'eval_steps_per_second': 7.798}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
#ref:https://huggingface.co/docs/peft/task_guides/lora_based_methods
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "k_lin", "v_lin"],
    #target_modules = ["score", "c_attn", "c_proj"], #not found in the base model
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

perf_model = get_peft_model(model, peft_config)
perf_model.print_trainable_parameters()



trainable params: 1,413,134 || all params: 67,776,014 || trainable%: 2.08500606128888


In [11]:
fine_tuned_trainer = Trainer(
    model=perf_model,
    args=TrainingArguments(
        output_dir="./data/peft-distilbert",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

fine_tuned_trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,1.433600,1.380911,0.599133
2,1.368500,1.379657,0.599133
3,1.384100,1.381476,0.599133
4,1.380600,1.380786,0.599133
5,1.383600,1.380623,0.599133


Checkpoint destination directory ./data/peft-distilbert/checkpoint-938 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft-distilbert/checkpoint-1876 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft-distilbert/checkpoint-2814 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft-distilbert/checkpoint-3752 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft-distilbert/checkpoint-4690 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4690, training_loss=1.3899295050452258, metrics={'train_runtime': 4079.2157, 'train_samples_per_second': 18.386, 'train_steps_per_second': 1.15, 'total_flos': 1.01242142208e+16, 'train_loss': 1.3899295050452258, 'epoch': 5.0})

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [13]:
# Saving the model the fine-tuned PEFT model
from peft import PeftModel

save_path = "./peft_model"
perf_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('./peft_model/tokenizer_config.json',
 './peft_model/special_tokens_map.json',
 './peft_model/vocab.txt',
 './peft_model/added_tokens.json',
 './peft_model/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
from peft import PeftModel, PeftConfig

# Load base model again
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=7)

# Reloading the model
peft_model = PeftModel.from_pretrained(base_model, "peft_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#compare the original and fine-tuned models
original_performance = trainer.evaluate()
fine_tuned_performance = fine_tuned_trainer.evaluate()

print("Original Model:", original_performance)
print()
print("Fine-Tuned Model:", fine_tuned_performance)


Original Model: {'eval_loss': 1.37965726852417, 'eval_accuracy': 0.5991333333333333, 'eval_runtime': 255.865, 'eval_samples_per_second': 58.625, 'eval_steps_per_second': 7.328}

Fine-Tuned Model: {'eval_loss': 1.37965726852417, 'eval_accuracy': 0.5991333333333333, 'eval_runtime': 261.7086, 'eval_samples_per_second': 57.316, 'eval_steps_per_second': 3.584, 'epoch': 5.0}


#### Using the same evaluation process (same metric(s) and dataset) we see a difference between the fine-tuned version and the original version of the model. The eval_accuracy': 0.3289336 earlier then later 0.59991 after fine tuning 